In [26]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-12-24 01:44:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4350 (4.2K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   4.25K  --.-KB/s    in 0s      

2025-12-24 01:44:28 (32.6 MB/s) - ‘minsearch.py.1’ saved [4350/4350]



In [27]:
import minsearch

/workspaces/llm-zoomcamp/01-intro/Section 1/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [28]:
import json

In [29]:
with open ('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [1]:
#docs_raw

In [31]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [32]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [33]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [12]:
q = 'the corse has already started, can I still enroll?'

In [34]:
index.fit(documents)

In [35]:
boost = { 'question' : 3.0, 'section': 0.5}
results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
    )

In [36]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
  'section': 'General course-related questions',
  'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'd

## Section 1.4

In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
import os

In [5]:
openai_api = os.environ['OPEN_AI_KEY']

In [10]:
client = OpenAI(api_key = openai_api)

In [13]:
q

'the corse has already started, can I still enroll?'

In [19]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content":q}]
)

In [22]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the policies of the institution offering the course. Here are a few steps you can take:\n\n1. **Contact the Institution:** Reach out to the admissions office or the department offering the course. They can provide you with specific information about late enrollment policies.\n\n2. **Talk to the Instructor:** If you get permission from the instructor, they may allow you to join the course even after it has begun, especially if it’s a short time after the start date.\n\n3. **Check Online Portals:** Some institutions allow late enrollment through their online systems, so check there for any options.\n\n4. **Consider Catching Up:** Be prepared to catch up on any missed content and assignments. Ask if there are resources available to help you get up to speed.\n\n5. **Review the Policies:** Make sure you understand any implications of late enrollment, such as potential late fees or impact on your academic record.\n

In [24]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the context doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

In [43]:
context = ""

for doc in results:
    context = context + f"section:{doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [44]:
print(context)

section:General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section:General course-related questions
question: Environment - Is Python 3.9 still the recommended version to use in 2024?
answer: Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]
But Python 3.10 and 3.11 should work fine.

section:General course-related questions
question: How can we contribute to the course?
answer: Star the repo! Share it with friends if you find it useful ❣️
Create a PR if you see you can improve the text or the structure of the repository.

section:General course-related questions
question: Are we still using the NYC Trip data for January 2021? Or are we using the 2022 data?
answer: We will use t

In [47]:
prompt = prompt_template.format(question=q, context = context).strip()

In [48]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the context doesn't contain the answer, output NONE

QUESTION: the corse has already started, can I still enroll?

CONTEXT:
section:General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section:General course-related questions
question: Environment - Is Python 3.9 still the recommended version to use in 2024?
answer: Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]
But Python 3.10 and 3.11 should work fine.

section:General course-related questions
question: How can we contribute to the course?
answer: Star the rep

In [49]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content":prompt}]
)

response.choices[0].message.content

"Yes, you can still join the course after the start date. You're eligible to submit the homeworks, but be aware of the deadlines for turning in the final projects."

## Section 1.5 

making code cleaner and more modular

In [51]:
def search(query): 
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results
    

In [79]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section:{doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context = context).strip()
    return prompt
    

In [74]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{"role": "user", "content":prompt}]
    )

    return response.choices[0].message.content
    

In [82]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [83]:
query = 'how do I run kafka?'
rag(query)

'To run Kafka in a Java project, you can execute producer, consumer, or kstreams in the terminal by navigating to the project directory and using the following command:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your jar file. This command will execute the file located at `src/main/java/org/example/JsonProducer.java`.'

In [84]:
rag('the course has already started, can I still enrol?')

'Yes, even if the course has already started, you can still enroll and are eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects.'

## Section 1.6

In [85]:
## Replace Minsearch with Elasticsearch
## He says Minsearch is toy